In [ ]:
import pandas as pd
import numpy as np
from lightfm import LightFM
from lightfm.data import Dataset
from lightfm.cross_validation import random_train_test_split
from lightfm.evaluation import precision_at_k, recall_at_k, auc_score

In [87]:
# Import Synthetic Data

USERS_PATH = "users_v3.csv"
ITEMS_PATH = "items_v3.csv"
INTERACTIONS_PATH = "interactions_v3.csv"

users = pd.read_csv(USERS_PATH)
items = pd.read_csv(ITEMS_PATH)
interactions = pd.read_csv(INTERACTIONS_PATH)

print("users:", users.shape)
print("items:", items.shape)
print("interactions:", interactions.shape)

display(users.head())
display(items.head())
display(interactions.head())


users: (1000, 17)
items: (16, 6)
interactions: (1881, 3)


,user_id,is_new_customer,has_internet,has_mobile,region,outage_risk,household_size,devices,iot_devices,wfh_count,gamer_count,creator_count,budget,monthly_data_gb,mobile_line_count,mobile_data_gb,current_mobile_bill
0,0,0,1,0,MW,2,1,7,2,0,1,0,54,649,1,10,54
1,1,0,1,0,NE,1,3,12,16,2,1,2,104,1734,2,44,66
2,2,0,1,0,NE,2,5,10,33,5,3,1,113,2500,4,69,120
3,3,0,1,0,NE,1,2,7,8,1,2,0,93,1194,1,22,24
4,4,0,1,0,NE,0,6,18,27,6,6,0,122,2500,6,70,170


,item_id,item_name,category,price,speed_mbps,notes
0,0,Internet 300,internet_tier,60,300.0,NaN
1,1,Internet 500,internet_tier,75,500.0,NaN
2,2,Internet 1000,internet_tier,95,1000.0,NaN
3,3,Internet 2000,internet_tier,120,2000.0,NaN
4,4,Mobile By-the-Gig,mobile_plan,25,NaN,NaN


,user_id,item_id,interaction_strength
0,0,0,3
1,1,3,3
2,1,8,3
3,2,2,3
4,2,7,3


In [89]:
# Bin the features to help LightFM

def bin_token(prefix: str, value: int, bins: list[tuple[int, int, str]]) -> str:
    for low, high, label in bins:
        if low <= value <= high:
            return f"{prefix}={label}"
    return f"{prefix}=unknown"

def count_bin(prefix: str, value: int) -> str:
    if value <= 0:
        return f"{prefix}=0"
    if value == 1:
        return f"{prefix}=1"
    return f"{prefix}=2+"

def outage_bin(value: int) -> str:
    if value <= 0:
        return "outage=low"
    if value == 1:
        return "outage=med"
    return "outage=high"

BUDGET_BINS = [
    (0, 59, "0-59"),
    (60, 79, "60-79"),
    (80, 119, "80-119"),
    (120, 9999, "120+"),
]

BB_DATA_BINS = [
    (0, 299, "0-299GB"),
    (300, 699, "300-699GB"),
    (700, 1199, "700-1199GB"),
    (1200, 99999, "1200GB+"),
]

MOBILE_DATA_BINS = [
    (0, 9, "0-9GB"),
    (10, 24, "10-24GB"),
    (25, 59, "25-59GB"),
    (60, 9999, "60GB+"),
]

IOT_BINS = [
    (0, 5, "0-5"),
    (6, 15, "6-15"),
    (16, 30, "16-30"),
    (31, 9999, "31+"),
]

DEV_BINS = [
    (0, 5, "0-5"),
    (6, 10, "6-10"),
    (11, 15, "11-15"),
    (16, 9999, "16+"),
]

MOBILE_BILL_BINS = [
    (0, 49, "0-49"),
    (50, 99, "50-99"),
    (100, 149, "100-149"),
    (150, 9999, "150+"),
]


In [90]:
# Create User Feature tokens for cold and warm starts

def build_user_feature_tokens(row: pd.Series, include_identity: bool = True) -> list[str]:
    """
    Builds user feature tokens.
    
    include_identity = True  -> warm start (existing dataset user)
    include_identity = False -> cold start (new external user)
    """

    feats = []

    # Only add identity feature if warm-start
    if include_identity:
        uid = int(row["user_id"])
        feats.append(f"user_id={uid}")

    # Categorical
    feats.append(f"region={row['region']}")
    feats.append(outage_bin(int(row["outage_risk"])))

    # Numeric binned
    feats.append(bin_token("budget", int(row["budget"]), BUDGET_BINS))
    feats.append(bin_token("bb_data", int(row["monthly_data_gb"]), BB_DATA_BINS))
    feats.append(bin_token("devices", int(row["devices"]), DEV_BINS))
    feats.append(bin_token("iot", int(row["iot_devices"]), IOT_BINS))

    # Household counts
    feats.append(count_bin("wfh", int(row["wfh_count"])))
    feats.append(count_bin("gamer", int(row["gamer_count"])))
    feats.append(count_bin("creator", int(row["creator_count"])))

    # Mobile context
    feats.append(f"has_mobile={int(row['has_mobile'])}")
    feats.append(count_bin("lines", int(row["mobile_line_count"])))
    feats.append(bin_token("m_data", int(row["mobile_data_gb"]), MOBILE_DATA_BINS))
    feats.append(bin_token("m_bill", int(row["current_mobile_bill"]), MOBILE_BILL_BINS))

    # Segment
    feats.append(f"is_new={int(row.get('is_new_customer', 1))}")

    return feats

user_features_map = {
    int(r.user_id): build_user_feature_tokens(pd.Series(r._asdict()), include_identity=True)
    for r in users.itertuples(index=False)
}

# peek at one user's features
some_uid = int(users.iloc[0]["user_id"])
print(some_uid, user_features_map[some_uid])


0 ['user_id=0', 'region=MW', 'outage=high', 'budget=0-59', 'bb_data=300-699GB', 'devices=6-10', 'iot=0-5', 'wfh=0', 'gamer=1', 'creator=0', 'has_mobile=0', 'lines=1', 'm_data=10-24GB', 'm_bill=50-99', 'is_new=0']


In [91]:
# create item feature tokens

def item_feature_tokens(row: pd.Series) -> list[str]:
    feats = []
    iid = int(row["item_id"])

    # Identity feature
    feats.append(f"item_id={iid}")

    # Category
    feats.append(f"category={row['category']}")

    # Price bins
    price = int(row["price"])
    feats.append(bin_token("price", price, [
        (0, 0, "0"),
        (1, 14, "1-14"),
        (15, 29, "15-29"),
        (30, 59, "30-59"),
        (60, 9999, "60+"),
    ]))

    # Speed bins for internet tiers
    if pd.notna(row["speed_mbps"]):
        speed = int(row["speed_mbps"])
        feats.append(bin_token("speed", speed, [
            (0, 399, "0-399"),
            (400, 799, "400-799"),
            (800, 1499, "800-1499"),
            (1500, 99999, "1500+"),
        ]))
    else:
        feats.append("speed=na")

    # Special note feature for storm-ready (optional)
    if isinstance(row.get("notes", None), str) and "power outage" in row["notes"].lower():
        feats.append("storm_ready=yes")

    return feats

item_features_map = {
    int(r.item_id): item_feature_tokens(pd.Series(r._asdict()))
    for r in items.itertuples(index=False)
}

# peek at one item
some_item = int(items.iloc[0]["item_id"])
print(some_item, item_features_map[some_item])


0 ['item_id=0', 'category=internet_tier', 'price=60+', 'speed=0-399']


In [92]:
# build lightfm data and factorization matrices

dataset = Dataset()

all_user_ids = users["user_id"].astype(int).tolist()
all_item_ids = items["item_id"].astype(int).tolist()

# gather unique feature tokens
all_user_feature_tokens = set()
for feats in user_features_map.values():
    all_user_feature_tokens.update(feats)

all_item_feature_tokens = set()
for feats in item_features_map.values():
    all_item_feature_tokens.update(feats)

dataset.fit(
    users=all_user_ids,
    items=all_item_ids,
    user_features=list(all_user_feature_tokens),
    item_features=list(all_item_feature_tokens),
)

# Build interactions (user_id, item_id, weight)
triples = list(zip(
    interactions["user_id"].astype(int),
    interactions["item_id"].astype(int),
    interactions["interaction_strength"].astype(float),
))

(interactions_matrix, weights_matrix) = dataset.build_interactions(triples)

user_features = dataset.build_user_features(
    [(uid, feats) for uid, feats in user_features_map.items()],
    normalize=False
)

item_features = dataset.build_item_features(
    [(iid, feats) for iid, feats in item_features_map.items()],
    normalize=False
)

print("Interactions matrix:", interactions_matrix.shape)
print("User features:", user_features.shape)
print("Item features:", item_features.shape)


Interactions matrix: (1000, 16)
User features: (1000, 2047)
Item features: (16, 48)


In [93]:
# Split data to train on 80% data

#from lightfm.cross_validation import random_train_test_split

train, test = random_train_test_split(
    interactions_matrix,
    test_percentage=0.2,
    random_state=np.random.RandomState(42)
)

print("Train nnz:", train.getnnz(), "| Test nnz:", test.getnnz())


Train nnz: 1504 | Test nnz: 377


In [94]:
# Train Model

model = LightFM(no_components=32,loss="warp",learning_rate=0.05,item_alpha=1e-6,user_alpha=1e-6,random_state=42)

model.fit(train,user_features=user_features,item_features=item_features,epochs=20,num_threads=4,verbose=True)

print("Training complete.")


Epoch 0
Epoch 1
Epoch 2
Epoch 3
Epoch 4
Epoch 5
Epoch 6
Epoch 7
Epoch 8
Epoch 9
Epoch 10
Epoch 11
Epoch 12
Epoch 13
Epoch 14
Epoch 15
Epoch 16
Epoch 17
Epoch 18
Epoch 19
Training complete.


In [153]:
# model performance evaluation

ks = [1, 3, 5, 10]

p = precision_at_k(
    model,
    test,
    train_interactions=train,
    user_features=user_features,
    item_features=item_features,
    k=k
).mean()

r = recall_at_k(
    model,
    test,
    train_interactions=train,
    user_features=user_features,
    item_features=item_features,
    k=k
).mean()

auc = auc_score(
    model,
    test,
    train_interactions=train,
    user_features=user_features,
    item_features=item_features
).mean()

item_pop = np.asarray(train.sum(axis=0)).ravel()
top_item_indices = np.argsort(-item_pop)[:k]

test_csr = test.tocsr()
hits = []
for u in range(test_csr.shape[0]):
    user_test_items = test_csr[u].indices
    if len(user_test_items) == 0:
        continue
    hits.append(any(i in top_item_indices for i in user_test_items))
    
f1 = 2*(p*r)/(p+r) if (p+r) > 0 else 0.0

print(f"Number of recommendations: {k:2d}")
print(f"Model Precision@{k}: {p:.4f}")
print(f"Model Recall@{k}: {r:.4f}")
print(f"Model F1@{k}: {f1:.4f}")
print(f"Model AUC: {auc:.4f}")
print(f"Popularity HitRate@{k}: {np.mean(hits):.4f} (rough baseline)")





Number of recommendations:  5
Model Precision@5: 0.2309
Model Recall@5: 0.9539
Model F1@5: 0.3718
Model AUC: 0.9090
Popularity HitRate@5: 0.7556 (rough baseline)


In [136]:
# -----------------------------
# Grouped recommendation helpers
# -----------------------------

# --- Dataset mappings (internal ids) ---
user_id_map, user_feature_map, item_id_map, item_feature_map = dataset.mapping()
inv_item_id_map = {v: k for k, v in item_id_map.items()}  # internal_item_idx -> item_id

items_by_id = items.set_index("item_id")

# mobile savings functions
def est_xm_cost_simple(user_row: dict, plan_name: str) -> int:
    lines = max(int(user_row["mobile_line_count"]), 1)
    if plan_name == "Unlimited":
        return 45 * lines
    if plan_name == "Unlimited+":
        return 60 * lines
    if plan_name == "By-the-Gig":
        return 30 * lines

    return 0


def best_plan_and_savings(user_row: dict):
    bill = int(user_row["current_mobile_bill"])
    plans = ["Unlimited", "Unlimited+", "By-the-Gig"]

    costs = {p: est_xm_cost_simple(user_row, p) for p in plans}
    best_plan = min(costs, key=costs.get)
    best_cost = costs[best_plan]
    savings = bill - best_cost

    return best_plan, best_cost, savings

# --- Core ranking (internal indices) ---
def _rank_item_internal_indices(
    user_internal: int,
    user_feat_matrix,
    n_pool: int = 10,
    exclude_seen: bool = False,
    seen_interactions=None,  # pass train CSR row if you want to exclude seen
):
    """
    Returns top internal item indices ranked by model score.
    Scores are not returned (we use ranking only).
    """
    all_item_internal = np.arange(len(item_id_map), dtype=np.int32)

    scores = model.predict(
        user_internal,
        all_item_internal,
        user_features=user_feat_matrix,
        item_features=item_features,
    )

    if exclude_seen and seen_interactions is not None:
        # seen_interactions must be a 1D list/array of internal item indices
        scores[seen_interactions] = -np.inf

    top_internal = np.argsort(-scores)[:n_pool]
    return top_internal


def _to_ranked_items_df(top_internal: np.ndarray) -> pd.DataFrame:
    """
    Converts ranked internal item indices to a ranked items DataFrame (no scores, no item_id).
    """
    ranked_item_ids = [int(inv_item_id_map[i]) for i in top_internal]  # item_id in original catalog
    df = items[items["item_id"].isin(ranked_item_ids)][["item_id", "item_name", "category"]].copy()

    # keep ranking order
    df["rank"] = df["item_id"].apply(lambda x: ranked_item_ids.index(int(x)))
    df = df.sort_values("rank").drop(columns=["rank", "item_id"]).reset_index(drop=True)  # drop item_id for UI
    return df


def _bucket_recommendations(
    ranked_df: pd.DataFrame,
    profile_for_savings: dict | None = None,
    max_addons: int = 3,
    max_mobile_ops: int = 3,
):
    """
    Buckets ranked recommendations into:
      - top (first recommendation)
      - addons
      - mobile_ops (bundle/offer/mobile_plan)
      - savings_note (optional explanation, only if offer appears)
    Returns DataFrames (no scores, no item_id).
    """
    if ranked_df.empty:
        empty = pd.DataFrame(columns=["item_name", "category"])
        return empty, empty, empty, None

    top = ranked_df.iloc[[0]].copy()

    addons = ranked_df[ranked_df["category"] == "addon"].head(max_addons).reset_index(drop=True)

    mobile_ops = ranked_df[
        ranked_df["category"].isin(["bundle", "offer", "mobile_plan"])
    ].head(max_mobile_ops).reset_index(drop=True)

    savings_note = None
    if profile_for_savings is not None:
        offer_present = ranked_df["item_name"].str.contains("Savings Offer", case=False, na=False).any()
        if offer_present and int(profile_for_savings.get("mobile_line_count", 0)) > 0:
            best_plan, best_cost, savings = best_plan_and_savings(profile_for_savings)
            savings_note = {
                "best_plan": best_plan,
                "estimated_cost_per_month": int(best_cost),
                "estimated_savings_per_month": int(savings),
            }

    return top, addons, mobile_ops, savings_note


def _display_grouped(top, addons, mobile_ops, savings_note=None):

    def _display_with_index_starting_at_1(df):
        if df is None or df.empty:
            display(pd.DataFrame({"item_name": []}))
            return
        df_display = df[["item_name"]].copy()
        df_display.index = np.arange(1, len(df_display) + 1)
        display(df_display)

    
    print("TOP RECOMMENDATION")
    _display_with_index_starting_at_1(top)

    print("\nRECOMMENDED ADD-ONS")
    _display_with_index_starting_at_1(addons)

    print("\nMOBILE / BUNDLE OPPORTUNITY")
    _display_with_index_starting_at_1(mobile_ops)

    if savings_note:
        print("\nESTIMATED MOBILE SAVINGS")
        print(
            f"Best plan: {savings_note['best_plan']} | "
            f"Est. cost: ${savings_note['estimated_cost_per_month']}/mo | "
            f"Est. savings: ${savings_note['estimated_savings_per_month']}/mo"
        )

# -----------------------------
# Warm-start: existing user
# -----------------------------
def recommend_existing_user_grouped(user_id: int, n_pool: int = 10, exclude_seen: bool = True):
    """
    Existing (warm-start) recommendations grouped for demo.
    """
    if user_id not in user_id_map:
        raise ValueError(f"user_id {user_id} not found in dataset.")

    u_internal = user_id_map[user_id]

    seen = None
    if exclude_seen:
        # uses training interactions if available
        try:
            seen = train.tocsr()[u_internal].indices
        except Exception:
            seen = None

    top_internal = _rank_item_internal_indices(
        user_internal=u_internal,
        user_feat_matrix=user_features,
        n_pool=n_pool,
        exclude_seen=exclude_seen,
        seen_interactions=seen,
    )

    ranked_df = _to_ranked_items_df(top_internal)
    top, addons, mobile_ops, _ = _bucket_recommendations(ranked_df, profile_for_savings=None)
    _display_grouped(top, addons, mobile_ops, savings_note=None)


# -----------------------------
# Cold-start: For new customer profile
# -----------------------------
def recommend_new_customer_grouped(profile: dict, n_pool: int = 10):
    """
    New customer (cold-start) recommendations grouped for demo.
    Uses inferred features (no identity feature).
    """
    row = pd.Series(profile)
    feats = build_user_feature_tokens(row, include_identity=False)

    # Anchor on any known user id so the dataset can build a feature row
    any_known_user_id = next(iter(user_id_map.keys()))
    any_known_user_internal = user_id_map[any_known_user_id]

    temp_user_features = dataset.build_user_features(
        [(any_known_user_id, feats)],
        normalize=False,
    )

    top_internal = _rank_item_internal_indices(
        user_internal=any_known_user_internal,
        user_feat_matrix=temp_user_features,
        n_pool=n_pool,
        exclude_seen=False,
        seen_interactions=None,
    )

    ranked_df = _to_ranked_items_df(top_internal)
    top, addons, mobile_ops, savings_note = _bucket_recommendations(ranked_df, profile_for_savings=profile)
    _display_grouped(top, addons, mobile_ops, savings_note=savings_note)


# -----------------------------
# Optional: flat (ungrouped) recommendations (no ids, no scores)
# -----------------------------
def recommend_existing_user_flat(user_id: int, n: int = 7, exclude_seen: bool = True):
    if user_id not in user_id_map:
        raise ValueError(f"user_id {user_id} not found in dataset.")

    u_internal = user_id_map[user_id]

    seen = None
    if exclude_seen:
        try:
            seen = train.tocsr()[u_internal].indices
        except Exception:
            seen = None

    top_internal = _rank_item_internal_indices(
        user_internal=u_internal,
        user_feat_matrix=user_features,
        n_pool=n,
        exclude_seen=exclude_seen,
        seen_interactions=seen,
    )
    ranked_df = _to_ranked_items_df(top_internal)
    return ranked_df[["item_name", "category"]]


def recommend_new_customer_flat(profile: dict, n: int = 7):
    row = pd.Series(profile)
    feats = build_user_feature_tokens(row, include_identity=False)

    any_known_user_id = next(iter(user_id_map.keys()))
    any_known_user_internal = user_id_map[any_known_user_id]

    temp_user_features = dataset.build_user_features([(any_known_user_id, feats)], normalize=False)

    top_internal = _rank_item_internal_indices(
        user_internal=any_known_user_internal,
        user_feat_matrix=temp_user_features,
        n_pool=n,
        exclude_seen=False,
        seen_interactions=None,
    )
    ranked_df = _to_ranked_items_df(top_internal)
    return ranked_df[["item_name", "category"]]


# -----------------------------
# Example users to test quickly
# -----------------------------
# existing_internet_only_uid = int(users[(users["is_new_customer"] == 0) & (users["has_mobile"] == 0)].iloc[0]["user_id"])
# existing_with_mobile_uid = int(users[(users["is_new_customer"] == 0) & (users["has_mobile"] == 1)].iloc[0]["user_id"])
# new_uid = int(users[users["is_new_customer"] == 1].iloc[0]["user_id"])

# print("Existing internet-only user:", existing_internet_only_uid)
# display(users[users["user_id"] == existing_internet_only_uid])
# display(recommend_existing_user_flat(existing_internet_only_uid, n=7))

# print("Existing with mobile user:", existing_with_mobile_uid)
# display(users[users["user_id"] == existing_with_mobile_uid])
# display(recommend_existing_user_flat(existing_with_mobile_uid, n=7))

# print("New (cold start) user (no interactions in data):", new_uid)
# display(users[users["user_id"] == new_uid])
# # NOTE: warm-start recommenders for truly new users are not meaningful unless you have features + cold-start path.




In [99]:
# import pandas as pd
# import numpy as np

# LightFM stores learned latent vectors here
user_emb = model.user_embeddings          # shape: [n_users_internal, n_components]
item_emb = model.item_embeddings          # shape: [n_items_internal, n_components]

# Save embeddings to CSV
pd.DataFrame(user_emb).to_csv("user_embeddings.csv", index=False)
pd.DataFrame(item_emb).to_csv("item_embeddings.csv", index=False)

print("Saved: user_embeddings.csv")
print("Saved: item_embeddings.csv")
print("user_embeddings shape:", user_emb.shape)
print("item_embeddings shape:", item_emb.shape)


Saved: user_embeddings.csv
Saved: item_embeddings.csv
user_embeddings shape: (2047, 32)
item_embeddings shape: (48, 32)


In [100]:
# Get mapping dictionaries so we can label rows/cols with your original IDs
user_id_map, user_feature_map, item_id_map, item_feature_map = dataset.mapping()
inv_user_id_map = {v: k for k, v in user_id_map.items()}
inv_item_id_map = {v: k for k, v in item_id_map.items()}

n_users_internal = len(user_id_map)
n_items_internal = len(item_id_map)

# Original IDs in internal order (so CSV is human-readable)
user_ids_in_order = [inv_user_id_map[u] for u in range(n_users_internal)]
item_ids_in_order = [inv_item_id_map[i] for i in range(n_items_internal)]

# Compute full score matrix
scores = np.zeros((n_users_internal, n_items_internal), dtype=np.float32)

all_item_internal = np.arange(n_items_internal, dtype=np.int32)
for u_internal in range(n_users_internal):
    scores[u_internal] = model.predict(
        u_internal,
        all_item_internal,
        user_features=user_features,
        item_features=item_features
    )

df_scores = pd.DataFrame(scores, index=user_ids_in_order, columns=item_ids_in_order)
df_scores.index.name = "user_id"
df_scores.to_csv("full_prediction_scores.csv")

print("Saved: full_prediction_scores.csv")
print("Scores shape:", df_scores.shape)


Saved: full_prediction_scores.csv
Scores shape: (1000, 16)


In [147]:
user_id_input = int(input("Enter user_id: "))
n_input = int(input("How many recommendations? "))

recommend_existing_user_grouped(user_id_input, n_input)

Enter user_id:  10
How many recommendations?  6


TOP RECOMMENDATION


,item_name
1,Internet 2000



RECOMMENDED ADD-ONS


,item_name
1,xFi Pro Extender (Storm-Ready)
2,xFi Complete
3,WiFi Extender



MOBILE / BUNDLE OPPORTUNITY


,item_name
1,Internet+Mobile Bundle


In [152]:
recommend_existing_user_grouped(10,6)

TOP RECOMMENDATION


,item_name
1,Internet 2000



RECOMMENDED ADD-ONS


,item_name
1,xFi Pro Extender (Storm-Ready)
2,xFi Complete
3,WiFi Extender



MOBILE / BUNDLE OPPORTUNITY


,item_name
1,Internet+Mobile Bundle


In [138]:
def collect_new_customer_simple_interactive():
    region = input("Region (NE/SE/MW/W): ")
    household_size = int(input("Household size: "))
    wfh_level = input("WFH level (none/some/heavy): ")
    gamer = input("Any gamers? (y/n): ").strip().lower() == "y"
    creator = input("Any content creators? (y/n): ").strip().lower() == "y"
    mobile_lines = int(input("Mobile lines: "))
    current_mobile_bill = int(input("Current mobile bill ($/mo): ")) if mobile_lines > 0 else 0
    budget = int(input("Monthly budget ($): "))

    return make_new_customer_profile(
        region=region,
        household_size=household_size,
        wfh_level=wfh_level,
        gamer=gamer,
        creator=creator,
        mobile_lines=mobile_lines,
        current_mobile_bill=current_mobile_bill,
        budget=budget
    )

profile = collect_new_customer_simple_interactive()
recommend_new_customer_grouped(profile, 5)

Region (NE/SE/MW/W):  SE
Household size:  4
WFH level (none/some/heavy):  some
Any gamers? (y/n):  n
Any content creators? (y/n):  y
Mobile lines:  5
Current mobile bill ($/mo):  200
Monthly budget ($):  300


TOP RECOMMENDATION


,item_name
1,Internet 2000



RECOMMENDED ADD-ONS


,item_name
1,xFi Complete
2,WiFi Extender
3,xFi Pro Extender (Storm-Ready)



MOBILE / BUNDLE OPPORTUNITY


,item_name
1,Internet+Mobile Bundle


In [149]:
profile = make_new_customer_profile(
    region="NE",
    household_size=5,
    wfh_level="heavy",
    gamer=True,
    creator=True,
    mobile_lines=5,
    current_mobile_bill=100,
    budget=300
)

#display(profile)
recommend_new_customer_grouped(profile, 5)


TOP RECOMMENDATION


,item_name
1,Internet 2000



RECOMMENDED ADD-ONS


,item_name
1,xFi Complete
2,WiFi Extender
3,xFi Pro Extender (Storm-Ready)



MOBILE / BUNDLE OPPORTUNITY


,item_name
1,Internet+Mobile Bundle
